In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
@import url('http://fonts.googleapis.com/css?family=Crimson+Text');
@import url('http://fonts.googleapis.com/css?family=Source+Code+Pro');

/* Change code font */
.CodeMirror pre {
    font-family: 'Input Mono Narrow', 'Source Code Pro', Consolas, monocco, monospace;
}

div.input_area {
    border-color: rgba(0,0,0,0.10);
}

div.text_cell {
    max-width: 105ex; /* instead of 100%, */
}

div.text_cell_render {
    font-family: "Crimson Text";
    font-size: 12pt;
    line-height: 145%; /* added for some line spacing of text. */
}

div.text_cell_render h1,
div.text_cell_render h2,
div.text_cell_render h3,
div.text_cell_render h4,
div.text_cell_render h5,
div.text_cell_render h6 {
    font-family: 'Crimson Text';
}

.rendered_html pre,
.rendered_html code {
    font-size: medium;
}

.rendered_html ol {
    list-style:decimal;
    margin: 1em 2em;
}

.prompt.input_prompt {
    color: rgba(0,0,0,0.5);
}

.cell.command_mode.selected {
    border-color: rgba(0,0,0,0.1);
}

.cell.edit_mode.selected {
    border-color: rgba(0,0,0,0.15);
    box-shadow: 0px 0px 5px #f0f0f0;
    -webkit-box-shadow: 0px 0px 5px #f0f0f0;
}

div.output_scroll {
    -webkit-box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
    box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
    border-radious: 2px;
}

#menubar .navbar-inner {
    -webkit-box-shadow: none;
    box-shadow: none;
    border-radius: 0;
    border: none;
    font-family: lato;
    font-weight: 400;
}

.navbar-fixed-top .navbar-inner,
.navbar-static-top .navbar-inner {
    box-shadow: none;
    -webkit-box-shadow: none;
    border: none;
}

div#notebook_panel {
    box-shadow: none;
    -webkit-box-shadow: none;
    border-top: none;
}

div#notebook {
    border-top: 1px solid rgba(0,0,0,0.15);
}

/* 
    This is a lazy fix, we *should* fix the 
    background for each Bootstrap button type
*/
#site * .btn {
    -webkit-box-shadow: none;
    box-shadow: none;
}


span.ansiblack {color: #073642;}
span.ansiblue {color: #2aa198;}
span.ansigray {color: #839496;}
span.ansigreen {color: #859900;}
span.ansipurple {color: #6c71c4;}
span.ansired {color: #dc322f;}
span.ansiyellow {color: #b58900;}

div.output_stderr {background-color: #dc322f;}
div.output_stderr pre {color: #eee8d5;}

.cm-s-ipython.CodeMirror {background: #fdf6e3; color: #073642;}
.cm-s-ipython div.CodeMirror-selected {background: #eee8d5 !important;}
.cm-s-ipython .CodeMirror-gutters {background: #fdf6e3; border-right: 0px;}
.cm-s-ipython .CodeMirror-linenumber {color: #839496;}
.cm-s-ipython .CodeMirror-cursor {border-left: 1px solid #657b83 !important;}

.cm-s-ipython span.cm-comment {color: #d33682;}
.cm-s-ipython span.cm-atom {color: #6c71c4;}
.cm-s-ipython span.cm-number {color: #6c71c4;}

.cm-s-ipython span.cm-property, .cm-s-ipython span.cm-attribute {color: #859900;}
.cm-s-ipython span.cm-keyword {color: #dc322f;}
.cm-s-ipython span.cm-string {color: #b58900;}
.cm-s-ipython span.cm-operator {color: #d33682;}
.cm-s-ipython span.cm-builtin {color: #6c71c4;}

.cm-s-ipython span.cm-variable {color: #859900;}
.cm-s-ipython span.cm-variable-2 {color: #268bd2;}
.cm-s-ipython span.cm-def {color: #cb4b16;}
.cm-s-ipython span.cm-error {background: #dc322f; color: #657b83;}
.cm-s-ipython span.cm-bracket {color: #586e75;}
.cm-s-ipython span.cm-tag {color: #dc322f;}
.cm-s-ipython span.cm-link {color: #6c71c4;}

.cm-s-ipython .CodeMirror-matchingbracket { text-decoration: underline; color: #073642 !important;}
th, td { 
      font-size: 18px;
      border-collapse: collapse;
      border-width:3px;
    }
    </style>

""")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.preprocessing import RobustScaler, QuantileTransformer
import rasterio
from matplotlib import colors
from scipy import stats
from tqdm import *
from pyproj import Transformer
import ast
import glob
from io import StringIO

| KPI Name                        | KPI Interpretation                                               | Public Data Source             |
|---------------------------------|------------------------------------------------------------------|--------------------------------|
| drought_severity                | drought severity index                                           | UNL/drougt.gov  (custom query) |
| drought_severity_variability    | drought variability index                                        | UNL/drougt.gov (custom query)  |
| water_loss                      | freshwater loss                                                  | SEDAC                          |
| all_locs_construction           | Ratio of construction workers in city compared to rest of the US | OECS                           |
| all_locs_farming                | Ratio of farming/ag workers in city compared to rest of the US   | OECS                           |
| score_drought                   | Calculated score based on survey response                        | CDP                            |
| score_wild_fire                 | Calculated score based on survey response                        | CDP                            |
| score_land_fire                 | Calculated score based on survey response                        | CDP                            |
| fema_drought_fire_events_ranked | # of FEMA drought / wild fire events disasters                   | FEMA National Risk Index       |
| drought_eals                    | drought expected annual loss                                     | FEMA National Risk Index       |
| wildfire_eals                   | wildfire expected annual loss                                    | FEMA National Risk Index       |

In [ ]:
path = "../input/externald/Supplementary Data/" 
vulnerability = pd.read_csv(path + "cities_updated_geo_us_2020.csv")
acct_number = vulnerability["Account Number"]
counties = (vulnerability['counties'])
counties_weights = (vulnerability['counties_weights'])
states = (vulnerability['state'])
flag_multi_counties = vulnerability['flag_multiple_counties']
match_fips = vulnerability['FIPS']
MSA = (vulnerability['MSA'])
match_fips = [ast.literal_eval(match_fips[i]) if flag_multi_counties[i] else int(match_fips[i]) for i in range(len(match_fips))]
cities = vulnerability["city_bing"]

In [ ]:
def standardize_rank(arr, direction=1):
    rank = (stats.mstats.rankdata(np.ma.masked_invalid(arr)))
    rank[rank == 0] = np.nan
    if(direction == 1):
        return rank/(np.nanmax(rank)) 
    else:
        return 1-(rank/(np.nanmax(rank)))


In [ ]:
fp = path + 'freshwater_availability.tif'
img = rasterio.open(fp)
img_array = img.read(1)

def get_value(raster, array, lon, lat):
    py, px = raster.index(lon, lat)
    return array[py, px]


In [ ]:
water_loss = []
for i in range(len(vulnerability)):
    coords = ((vulnerability['loc'].iloc[i].replace("POINT (", "")).replace(")","")).split()
    lon = float(coords[0])
    lat = float(coords[1])
    water_loss.append(get_value(img, img_array, lon, lat))

In [ ]:
water_loss_ranked = standardize_rank(water_loss, direction=-1)

In [ ]:
drh = pd.read_csv(path + "drought_monitor_percent_by_population_categorical_2010_2020.csv")
drh["Date"] = pd.to_datetime(drh["MapDate"], format="%Y%m%d")

In [ ]:
def get_metric_county_year(county, year):
    cdf = drh[(drh['Date'].dt.year == year) & (drh['FIPS'] == county)]
    val = np.array(1.0*(cdf['D0']) + 3.0*(cdf['D1']) + 5.0*(cdf['D2']) + 10.0*(cdf['D3']) + 20.0*(cdf['D4']))
    return val

In [ ]:
allc = []
for j in tqdm(range(len(match_fips))):
    years = np.arange(2010, 2021)
    combine = []
    for i in range(len(years)):
        if(not flag_multi_counties[j]):
            val = get_metric_county_year(match_fips[j], years[i])
            argsortval = (np.argsort(val))[::-1]
            val = np.mean(val[argsortval[0:4]])
            combine.append(val)
        else:
            temp = []
            for k in range(len(match_fips[j])):
                val = get_metric_county_year(match_fips[j][k], years[i])
                argsortval = (np.argsort(val))[::-1]
                val = np.mean(val[argsortval[0:4]])
                temp.append(val)
            ar = np.array(temp)
            weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[j])))
            combine.append(weighted_avg)
    allc.append(combine)

In [ ]:
last_decade_all_cities = np.array(allc)
dsci = np.median(last_decade_all_cities, axis=1)
dsci_ranked =  standardize_rank(dsci, direction=1)
dsci_std = np.std(last_decade_all_cities, axis=1)
dsci_std_ranked =  standardize_rank(dsci_std, direction=1)

In [ ]:
occupation = pd.read_excel(path + "Occupation_Data_OECS/MSA_M2019_dl.xlsx", sheet_name="All May 2019 Data")

all_locs_farming = []
for i in range(len(MSA)):
    o = (occupation[occupation["area"] == MSA[i]])
    locs = o[o['occ_code'].str.startswith("45")]["loc_quotient"]
    locs = np.array([float(locs.iloc[x]) for x in range(len(locs)) if type(locs.iloc[x]) is float])
    all_locs_farming.append(np.nanmean(locs))
    
all_locs_construction = []
for i in range(len(MSA)):
    o = (occupation[occupation["area"] == MSA[i]])
    locs = o[o['occ_code'].str.startswith("47")]["loc_quotient"]
    locs = np.array([float(locs.iloc[x]) for x in range(len(locs)) if type(locs.iloc[x]) is float])
    all_locs_construction.append(np.nanmean(locs))

In [ ]:
all_locs_farming_ranked = standardize_rank(all_locs_farming, 1)
all_locs_construction_ranked = standardize_rank(all_locs_construction, 1)


In [ ]:
cities_2020 = pd.read_csv("/kaggle/input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv")

cities_disc_2020 = pd.read_csv("/kaggle/input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv")

cities_2020_merged = pd.merge(cities_2020, cities_disc_2020, on="Account Number", how="outer", copy=False)
unique_20 = np.unique(cities_disc_2020[cities_disc_2020["Country"] == "United States of America"]["Account Number"])
cities_2020_merged_sub = cities_2020_merged[np.in1d(cities_2020_merged["Account Number"], unique_20)]

def fetch_answer_individual(account, question_number, org_type="city", year=2018, corp_res_type=None, column_number=None, row_number=None):
    if(org_type=="city"):
        df = cities_2020_merged_sub
        subset = df[df['Question Number'] == question_number]
        answer = subset[subset["Account Number"] == account]
        if((column_number is not None) and (row_number is None)):
            answer = answer[answer["Column Number"] == column_number]
        elif((column_number is not None) and (row_number is not None)):
            answer = answer[(answer["Column Number"] == column_number) & (answer["Row Number"] == row_number)]
        elif((column_number is None) and (row_number is not None)):
            answer = answer[answer["Row Number"] == row_number]
        else:
            pass
    elif(org_type=="corp" and corp_res_type=="cc"):
        df = corporations_2020
        subset = df[df['question_number'] == question_number]
        answer = subset[subset["account_number"] == account]
        if((column_number is not None) and (row_number is None)):
            answer = answer[answer["column_number"] == column_number]
        elif((column_number is not None) and (row_number is not None)):
            answer = answer[(answer["column_number"] == column_number) & (answer["row_number"] == row_number)]
        elif((column_number is None) and (row_number is not None)):
            answer = answer[answer["row_number"] == row_number]
        else:
            pass
    elif(org_type=="corp" and corp_res_type=="ws"):
        df = all_corps_cc[year]
        subset = df[df['question_number'] == question_number]
        answer = subset[subset["account_number"] == account]
    else:
        print("Something went wrong. Try again.")
    
    return answer

In [ ]:
scores_lo_hi = {
    "Low" : 1, 
    "Medium Low": 2, 
    "Medium":3,
    "Medium High":4,
    "High":5,
    "Do not know":0.5,
    "Does not currently impact the city":0.5,
    "nan":0,
}

scores_increasing_decreasing = {
    "Increasing" : 2,
    "Decreasing" : 0.5,
    "None" : 1, 
    "Do not know" : 1, 
    "Not expected to happen in the future" : 0.1,
    "nan" : 0.0
}

def score_hazard_risk(cdf):
    prob = scores_lo_hi[str(cdf[cdf["Column Number"] == 3.0]["Response Answer"].iloc[0])]
    mag = scores_lo_hi[str(cdf[cdf["Column Number"] == 4.0]["Response Answer"].iloc[0])]
    
    future_freq = scores_increasing_decreasing[str(cdf[cdf["Column Number"] == 8.0]["Response Answer"].iloc[0])]
    future_intensity = scores_increasing_decreasing[str(cdf[cdf["Column Number"] == 9.0]["Response Answer"].iloc[0])]
    
    future_mag = scores_lo_hi[str(cdf[cdf["Column Number"] == 10.0]["Response Answer"].iloc[0])]
    return (prob*mag) + (prob*future_freq*future_intensity*future_mag)

def return_risk_and_score_per_county(hazard_type):
    risk = []
    self_reported_score = []

    for i in range(len(vulnerability)):
        df = fetch_answer_individual(vulnerability["Account Number"].iloc[i], '2.1')
        idx = np.where(np.array(df["Response Answer"]) == hazard_type)[0]
        if(len(idx) > 0):
            risk.append(1)
            cdf = df[df["Row Number"] == df["Row Number"].iloc[idx[0]]]
            self_reported_score.append(score_hazard_risk(cdf))
        else:
            risk.append(np.nan)
            self_reported_score.append(np.nan)
    return risk, self_reported_score


In [ ]:
risk_drought, score_drought = return_risk_and_score_per_county("Water Scarcity > Drought")
risk_wild_fire, score_wild_fire = return_risk_and_score_per_county("Wild fire > Forest fire")
risk_land_fire, score_land_fire = return_risk_and_score_per_county("Wild fire > Land fire")

In [ ]:
drought_ranked = standardize_rank(score_drought, direction=1)
wild_fire_ranked = standardize_rank(score_wild_fire, direction=1)
land_fire_ranked = standardize_rank(score_land_fire, direction=1)

In [ ]:
fema = pd.read_csv("../input/femadisasters/DisasterDeclarationsSummaries.csv")
ct = np.char.zfill(np.array(fema['fipsCountyCode']).astype(str), 3)
st = np.char.zfill(np.array(fema['fipsStateCode']).astype(str), 2)
fema['fips'] = np.core.defchararray.add(st, ct).astype(int)

fema_drought_fire_events = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        cdf = fema[(fema["incidentType"] == "Fire") | (fema["incidentType"] == "Drought")]
        cdf = cdf[cdf["fips"] == fips]
        fema_drought_fire_events.append(len(cdf))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = fema[(fema["incidentType"] == "Fire") | (fema["incidentType"] == "Drought")]
            cdf = cdf[cdf["fips"] == fips[j]]
            temp.append(len(cdf))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        fema_drought_fire_events.append(weighted_avg)


In [ ]:
fema_drought_fire_events_ranked = standardize_rank(fema_drought_fire_events, direction=1)

In [ ]:
score_severity = {
    "Extremely serious": 6,
    "Serious": 3,
    "Less Serious": 1
}

score_prob = {
    "Low" : 1, 
    "Medium-low": 2, 
    "Medium":3,
    "Medium-high":4,
    "High":5,
    "Do not know":0.5,
    "Does not currently impact the city":0.5,
    "nan":0,
}

def score_water_risk(acct_number):
    sub = fetch_answer_individual(acct_number, '14.2a')
    number_of_rows = len(np.unique(sub["Row Number"]))
    flag = np.any(sub["Response Answer"].str.contains("Question not applicable"))
    if((number_of_rows > 0) & (not flag)):
        mag = np.array(sub[sub["Column Number"] == 3.0]["Response Answer"])
        score_mag = sum([score_severity[mag[i]] if type(mag[i]) == str else 1 for i in range(len(mag))])
        prob = np.array(sub[sub["Column Number"] == 4.0]["Response Answer"])
        score_probab = sum([score_prob[prob[i]] if type(prob[i]) == str else 1 for i in range(len(prob))])
        return number_of_rows*score_mag*score_probab
    else:
        return 0

In [ ]:
water_security_risk_scores = [score_water_risk(acct_number[i]) for i in range(len(acct_number))]
water_security_risk_ranked = standardize_rank(water_security_risk_scores, direction=1)

In [ ]:
nri = pd.read_csv(path + "NRI_Table_Counties.csv")

def nri_select(field):
    res = []
    for i in range(len(match_fips)):
        if(not flag_multi_counties[i]):
            fips = int(match_fips[i])
            cdf = nri[nri["STCOFIPS"] == fips]
            try:
                res.append(float(cdf[field]))
            except:
                res.append(np.nan)
        else:
            fips = match_fips[i]
            temp = []
            for j in range(len(fips)):
                cdf = nri[nri["STCOFIPS"] == fips[j]]
                try:
                    temp.append(float(cdf[field]))
                except:
                    temp.append(np.nan)
            ar = np.array(temp)
            weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
            res.append(weighted_avg)
    return np.array(res)

drought_eals = nri_select("DRGT_EALS")
wildfire_eals = nri_select("WFIR_EALS")

In [ ]:
drought_eals_ranked = standardize_rank(drought_eals, 1)
wildfire_eals_ranked = standardize_rank(wildfire_eals, 1)

In [ ]:
vulnerability_raw = vulnerability.copy()
vulnerability_ranked = vulnerability.copy()
vulnerability_aggr = vulnerability.copy()

In [ ]:
### EXPOSURE 

vulnerability_raw["water_loss"] = water_loss
vulnerability_raw["drought_severity_index"] = dsci
vulnerability_raw["drought_severity_index_variability"] = dsci_std

vulnerability_raw["score_drought"] = score_drought
vulnerability_raw["score_wild_fire"] = score_wild_fire
vulnerability_raw["score_land_fire"] = score_land_fire
vulnerability_raw["water_security_risk_score"] = water_security_risk_scores

vulnerability_raw["fema_drought_fire_events"] = fema_drought_fire_events

### SENSITIVITY 

vulnerability_raw["LOC_farming_fishing_forestry"] = all_locs_farming
vulnerability_raw["LOC_construction"] = all_locs_construction

vulnerability_raw["drought_expected_annual_loss"] = drought_eals
vulnerability_raw["wildfire_expected_annual_loss"] = wildfire_eals
vulnerability_raw.to_csv("vulnerability_cities_us_drought_kpis_raw.csv")

In [ ]:
### EXPOSURE 

vulnerability_ranked["water_loss_ranked"] = water_loss_ranked
vulnerability_ranked["drought_severity_index_ranked"] = dsci_ranked
vulnerability_ranked["drought_severity_index_variability_ranked"] = dsci_std_ranked

vulnerability_ranked["drought_ranked"] = drought_ranked
vulnerability_ranked["wild_fire_ranked"] = wild_fire_ranked
vulnerability_ranked["land_fire_ranked"] = land_fire_ranked
vulnerability_ranked["water_security_risk_ranked"] = water_security_risk_ranked

vulnerability_ranked["fema_drought_fire_events_ranked"] = fema_drought_fire_events_ranked

### SENSITIVITY 

vulnerability_ranked["LOC_farming_fishing_forestry_ranked"] = all_locs_farming_ranked
vulnerability_ranked["LOC_construction_ranked"] = all_locs_construction_ranked

vulnerability_ranked["drought_expected_annual_loss_ranked"] = drought_eals_ranked
vulnerability_ranked["wildfire_expected_annual_loss_ranked"] = wildfire_eals_ranked
vulnerability_ranked.to_csv("vulnerability_cities_us_drought_kpis_ranked.csv")

In [ ]:
def aggr_and_rescale(arr):
    avg_arr = np.nanmean(arr, axis=1)
    rescaled_avg_arr = standardize_rank(avg_arr)
    return rescaled_avg_arr

all_expo_drought_kpis = np.c_[water_loss_ranked, dsci_ranked, dsci_std_ranked, drought_ranked, wild_fire_ranked, land_fire_ranked, 
                     water_security_risk_ranked, fema_drought_fire_events_ranked]

aggr_expo_drought_kpis = aggr_and_rescale(all_expo_drought_kpis)

all_sens_drought_kpis = np.c_[all_locs_farming_ranked, all_locs_construction_ranked, drought_eals_ranked, wildfire_eals_ranked]

aggr_sens_drought_kpis = aggr_and_rescale(all_sens_drought_kpis)

aggr_drought_kpi = aggr_and_rescale(np.c_[aggr_expo_drought_kpis, aggr_sens_drought_kpis])

In [ ]:
vulnerability_aggr = vulnerability.copy()

vulnerability_aggr["aggr_drought_kpis"] = aggr_drought_kpi
vulnerability_aggr["aggr_exposure_drought_kpis"] = aggr_expo_drought_kpis
vulnerability_aggr["aggr_sensitivity_drought_kpis"] = aggr_sens_drought_kpis

vulnerability_aggr.to_csv("vulnerability_cities_us_drought_aggr_kpis.csv")